In [1]:
from src.managers.postgres_checkpoint_manager import PostgresCheckpointerManager
from src.core.conversation_graph import ConversationGraph, ConversationProcessor
from src.utils import get_llm

from src.managers.postgres_db_manager import PostgresDBManager
from src.managers.reminder_manager import ReminderManager

In [2]:
llm_fo = get_llm()
conversation_graph = ConversationGraph(llm=llm_fo).compile()
processor = ConversationProcessor(conversation_graph)

exceuted postgres setup


In [3]:
from src.managers.user_manager import UserManager
um = UserManager()

In [4]:
# um.add_user(name='Rakesh', age_range='18-24', preferred_activities= ["Movies",'Swimming', "Gym"])

um.get_user('60d53be5-82cd-4511-9344-6d451eee459f')

{'user_id': '60d53be5-82cd-4511-9344-6d451eee459f',
 'name': 'Rakesh',
 'age_range': '18-24',
 'preferred_activities': ['Movies', 'Swimming', 'Gym'],
 'created_at': datetime.datetime(2025, 3, 11, 13, 54, 26, 803604)}

In [5]:
user_id = '60d53be5-82cd-4511-9344-6d451eee459f'
thread_id = 'test_thread_4'

In [6]:
conversation_graph.update_state(
                config={'configurable': {'thread_id': thread_id, 'user_id': user_id}},
                values={'user_info': um.get_user(user_id), 'flow': 'activity_suggestion'}
            )

{'configurable': {'thread_id': 'test_thread_4',
  'checkpoint_ns': '',
  'checkpoint_id': '1effe6ed-a4e7-6cbf-806d-2a872d4eec13'}}

In [8]:
response = processor.process_input("Hello", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='Initial greeting, no indication of crisis or harmful behavior' <=====
You: Hello
Sakha: Hello Rakesh, I'm Sakha, your friendly companion. How's your day going so far?


In [9]:
processor.process_input("Its overwhelmed with work", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is overwhelmed with work, no indication of suicidal tendencies or harmful behavior' <=====
You: Its overwhelmed with work
Sakha: I can totally understand, Rakesh. It can be really tough to feel overwhelmed with work. Would you like to take a break and do something you enjoy? Maybe we could plan something fun for later, like watching a movie or going for a swim?


In [13]:
processor.process_input("break sounds like a good idea but i'm in office", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is overwhelmed with work but open to taking a break, no indication of suicidal tendencies or harmful behavior.' <=====
You: break sounds like a good idea but i'm in office
Sakha: Sometimes, a short break can be really helpful. Why don't you take a few minutes to stretch, grab a snack or drink, or simply step outside for some fresh air? If you're feeling up to it, we could also plan something for after work, like a trip to the gym or a swim. What do you think?


In [7]:
cs = conversation_graph.get_state(
    config={'configurable': {'thread_id': thread_id, 'user_id': user_id}}
)

chs = cs.values.get('conversation_history')
chs2 = chs[:6]
conversation_graph.update_state(
    config={'configurable': {'thread_id': thread_id, 'user_id': user_id}},
    values={'conversation_history': chs2}
)

{'configurable': {'thread_id': 'test_thread_4',
  'checkpoint_ns': '',
  'checkpoint_id': '1effe6ed-c781-64f5-806e-01eddbe3a2e0'}}

In [8]:
processor.process_input("walk sounds good", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is responding positively to suggestions for a break and showing interest in a walk, no indication of suicidal tendencies or harmful behavior' <=====
You: walk sounds good
{'reply': 'A walk sounds like a great idea, Rakesh! Getting some fresh air and stretching your legs can really help clear your mind. Shall we plan a 15-20 minute walk for after work? I can also set a reminder for you, would you like that?', 'activity_details': None}
Sakha: A walk sounds like a great idea, Rakesh! Getting some fresh air and stretching your legs can really help clear your mind. Shall we plan a 15-20 minute walk for after work? I can also set a reminder for you, would you like that?


In [9]:
processor.process_input("after work is too late, i want to do it after 10 mins", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User wants to take a short walk after 10 minutes, no indication of crisis or harmful intentions' <=====
You: after work is too late, i want to do it after 10 mins
Reminder for walk at 17:29 for 15 is scheduled!
{'reply': "A short walk after 10 minutes sounds like a great idea, Rakesh! I'll set a reminder for you. Just to confirm, you'd like to go for a 15-20 minute walk in 10 minutes, is that right?", 'activity_details': {'activity': 'walk', 'time': '17:29', 'duration': 15}}
Sakha: A short walk after 10 minutes sounds like a great idea, Rakesh! I'll set a reminder for you. Just to confirm, you'd like to go for a 15-20 minute walk in 10 minutes, is that right?
{'activity': 'walk', 'time': '17:29', 'duration': 15}


In [10]:
processor.process_input("yes it is", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='The user has confirmed their walk plan and there are no indications of harmful intentions or suicidal tendencies.' <=====
You: yes it is
Reminder for walk at 17:30 for 15 is scheduled!
{'reply': "Great, I've set a reminder for your 15-20 minute walk in 10 minutes. You can take a break and come back feeling refreshed!", 'activity_details': {'activity': 'walk', 'time': '17:30', 'duration': 15}}
Sakha: Great, I've set a reminder for your 15-20 minute walk in 10 minutes. You can take a break and come back feeling refreshed!
{'activity': 'walk', 'time': '17:30', 'duration': 15}


In [11]:
processor.process_input("great tq", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is thanking and no crisis is apparent' <=====
You: great tq
Reminder for walk at 17:30 for 15 is scheduled!
{'reply': "You're welcome, Rakesh! I'll remind you to go for your walk in 10 minutes. Have a great time", 'activity_details': {'activity': 'walk', 'time': '17:30', 'duration': 15}}
Sakha: You're welcome, Rakesh! I'll remind you to go for your walk in 10 minutes. Have a great time
{'activity': 'walk', 'time': '17:30', 'duration': 15}


In [16]:
#reminder flow

In [18]:
conversation_graph.update_state(
            config={'configurable': {'thread_id': thread_id, 'user_id': user_id}},
            values={'flow': 'reminder', 'exchange': 0}
        )
sys_input = "send a reminder to user for activity"
response = processor.process_input("", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='no harmful intentions detected and user previously engaged in normal conversation' <=====
You: 
{'reply': "It's time for your walk, Rakesh! I'm really excited for you to take a break and get some fresh air. Remember, this is your time to relax and recharge. You got this! Take a deep breath, and let's get moving! ", 'activity_details': None}
Sakha: It's time for your walk, Rakesh! I'm really excited for you to take a break and get some fresh air. Remember, this is your time to relax and recharge. You got this! Take a deep breath, and let's get moving! 


SyntaxError: 'return' outside function (370462770.py, line 7)

In [19]:
processor.process_input("I'm going for the walk, appreciate the reminder", thread_id=thread_id, user_id=user_id)

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is going for a walk and appreciates the reminder, no indication of crisis or harmful intentions' <=====
You: I'm going for the walk, appreciate the reminder
{'reply': "That's great to hear, Rakesh! I'm glad I could remind you to take a break and go for your walk. Have a wonderful time and don't hesitate to reach out when you're back. I'd love to hear about your experience!", 'activity_details': None}
Sakha: That's great to hear, Rakesh! I'm glad I could remind you to take a break and go for your walk. Have a wonderful time and don't hesitate to reach out when you're back. I'd love to hear about your experience!


In [5]:
user_input="Hello"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User input is a greeting, no indication of crisis or harmful intentions' <=====
You: Hello
Frienn: Hello! I'm Frienn, your friendly companion. How's your day been so far?


In [6]:
user_input="I think it was great"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User reported a great day with no indication of crisis or harmful behavior.' <=====
You: I think it was great
Frienn: That's wonderful to hear! What made your day so great?


In [7]:
user_input="Completed long pending tasks"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is sharing positive experience of completing tasks' <=====
You: Completed long pending tasks
Frienn: That's amazing! Completing long pending tasks can be such a great feeling. You must be relieved and proud of yourself. What are you looking forward to doing next?


In [11]:
user_input="celebrate"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User wants to celebrate and is in a positive mood' <=====
You: celebrate
Frienn: That's a great idea! Celebrating your accomplishments can be a wonderful way to acknowledge your hard work. Would you like to do something special to celebrate, like going for a walk, or would you like me to suggest something?


In [12]:
user_input="walk is not for celebration, suggest something crazy"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is looking for celebration ideas, no indication of crisis or harm' <=====
You: walk is not for celebration, suggest something crazy
Frienn: How about trying a cooking class, going to a trampoline park, or even an escape room? These activities can add a bit of excitement to your celebration.


In [5]:
user_input="i want to complete celebration today"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User wants to complete celebration today, no indication of crisis or harmful intentions' <=====
You: i want to complete celebration today
Frienn: Let's plan to complete your celebration by 11 PM tonight. How about we allocate 2 hours for it? We can set a reminder for 11 PM to ensure you have enough time for your celebration.


In [7]:
user_input="sounds good, but what to do?"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is looking for celebration ideas and is not expressing any harmful intentions' <=====
You: sounds good, but what to do?
Frienn: How about trying a cooking class or going to a trampoline park? You could also consider an escape room. Which one of these activities sounds interesting to you?


In [8]:
user_input="None, sometthing indoors"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is looking for indoor activities to improve their mood' <=====
You: None, sometthing indoors
Frienn: How about indoor bowling, pool, or board games?


In [9]:
user_input="None of your suggestions are good, a movie sounds better"
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is expressing displeasure with suggestions but is still engaged and looking for alternative activities to improve their mood' <=====
You: None of your suggestions are good, a movie sounds better
Frienn: A movie sounds like a great idea! What kind of movie are you in the mood for? And how about we plan for a 2-hour movie session?


In [11]:
user_input="bye i'll enjoy my action movie "
processor.process_input(user_input, thread_id="133", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is in a good mood and has found an activity to enjoy' <=====
You: bye i'll enjoy my action movie 
Reminder for watching action movie scheduled!
Frienn: Have a blast watching your action movie! Enjoy every moment of it. I'll talk to you later, have a great time and take care!


In [10]:
cs3 = conversation_graph.update_state(config={'configurable':{'thread_id':"1", 'user_id':"dev-user"}}, values = {'flow':'reminder', 'exchange':0})

In [11]:
user_input=""
processor.process_input(user_input, thread_id="1", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='No indication of crisis or harmful intentions, previous conversation was about stress management and planning a walk.' <=====
You: 
Frienn: replyToUser="It's time to start your walk! Get some fresh air, clear your mind, and boost your energy. You got this! I'll be here when you're back, ready to hear about your experience." suggestAlternatives=False


In [12]:
user_input="i'm starting now"
processor.process_input(user_input, thread_id="1", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='user is starting their walk and no indication of crisis or harmful intentions' <=====
You: i'm starting now
Frienn: replyToUser="You're taking the first step towards a refreshing morning! Remember, it's just 30 minutes, and you can always listen to music or a podcast while you walk to make it more enjoyable. Have a great walk!" suggestAlternatives=False


In [13]:
user_input="hey do you remember me?"
processor.process_input(user_input, thread_id="1", user_id="dev-user-2")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='The user is asking a non-crisis related question, inquiring if they are remembered, which seems to be a casual conversation starter.' <=====
You: hey do you remember me?
Frienn: replyToUser="Of course, I remember you! You were just about to go for a walk to refresh your mind before work. How's it going so far?" suggestAlternatives=False


In [14]:
user_input="hey do you remember me?"
processor.process_input(user_input, thread_id="2", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is asking about memory, no indication of crisis or harmful behavior.' <=====
You: hey do you remember me?
Frienn: replyToUser="Hey, I'm Frienn, your friendly virtual companion. I'm here to listen and support you. I don't have personal memories, but I can access our previous conversations to make our chat more personal. How are you doing today?" didUserAgreeOnActivity=False didUserAgreeOnTime=False didUserAgreeOnDuration=False reminder=None


In [15]:
cs3 = conversation_graph.update_state(config={'configurable':{'thread_id':"1", 'user_id':"dev-user"}}, values = {'flow':'follow-up', 'exchange':0})
conversation_graph.get_state(config={'configurable':{'thread_id':"1", 'user_id':"dev-user"}}).values

{'exchange': 0,
 'conversation_history': [HumanMessage(content='Hey.', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hi, I'm Frienn, your friendly companion. How's your day starting out?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="I'm expecting to get overwhelmed with my work", additional_kwargs={}, response_metadata={}),
  AIMessage(content="I can sense that you're feeling a bit stressed about work. Would you like to take a short walk or listen to some music to help clear your mind before diving in?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='sounds good, my work starts at 9:30 am', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Why don't you take a 30-minute walk from 8:45 am to 9:15 am to refresh your mind before work? I can set a reminder for you.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="sure, I'll do that but will start at 8:36 am", additional_kwargs={}, respo

In [17]:
user_input=""
processor.process_input(user_input, thread_id="1", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='no harmful intentions detected' <=====
You: 
Frienn: replyToUser="I'm so glad you went for that walk! How did it make you feel? Did it help you clear your mind before work?" isFeedbackCollectionComplete=True activityFeedback=ActivityFeedback(activity='walking', completed=True, enjoyment_score=4, reason_skipped=None)


In [18]:
user_input="Nah, roads were full of vehicles, smoke and dust. People were rushing to offices. I was better before"
processor.process_input(user_input, thread_id="1", user_id="dev-user")

=====> Supervisor Decision: pickedRoute='continue_chat' reason='User is expressing frustration but no indication of suicidal tendencies or harmful behavior.' <=====
You: Nah, roads were full of vehicles, smoke and dust. People were rushing to offices. I was better before
Frienn: replyToUser="I'm so sorry to hear that your walk wasn't as refreshing as you had hoped. The traffic and pollution can be really overwhelming. It's completely okay to feel that way, and I'm here to listen. Would you like to talk about what's on your mind or is there anything else I can do to support you?" isFeedbackCollectionComplete=True activityFeedback=ActivityFeedback(activity='walking', completed=True, enjoyment_score=1, reason_skipped=None)
